In [ ]:
#default_exp cnn

In [ ]:
#hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# CNN


> This module contains all the code for training and extracting features from CNNs.

In [ ]:
#export
from fastai.vision import *
from fastai.metrics import error_rate
import torch
import torch.nn as nn
from torchvision import models
from torch.autograd import Variable
import cv2
from nbdev.showdoc import *

In [ ]:
#export
class Hook():
    "Create a hook on `m` with `hook_func`."
    def __init__(self, m:nn.Module, hook_func:HookFunc, is_forward:bool=True, detach:bool=True):
        self.hook_func,self.detach,self.stored = hook_func,detach,None
        f = m.register_forward_hook if is_forward else m.register_backward_hook
        self.hook = f(self.hook_fn)
        self.removed = False

    def hook_fn(self, module:nn.Module, input:Tensors, output:Tensors):
        "Applies `hook_func` to `module`, `input`, `output`."
        if self.detach:
            input  = (o.detach() for o in input ) if is_listy(input ) else input.detach()
            output = (o.detach() for o in output) if is_listy(output) else output.detach()
        self.stored = self.hook_func(module, input, output)

    def remove(self):
        "Remove the hook from the model."
        if not self.removed:
            self.hook.remove()
            self.removed=True

    def __enter__(self, *args): return self
    def __exit__(self, *args): self.remove()

In [ ]:
#export
def get_output(module, input_value, output):
    return output.flatten(1)

def get_input(module, input_value, output):
    return list(input_value)[0]

def get_named_module_from_model(model, name):
    for n, m in model.named_modules():
        if n == name:
            return m
    return None

In [ ]:
#export
def trainPetsModel():
    '''Fine tunes a pretrained ResNet50 model to detect pet breeds 
    and returns the resulting model in learn as well as the last layer
    in linear_output_layer for subsequent feature extraction.'''
    
    bs = 16
    np.random.seed(2)
    
    path = untar_data(URLs.PETS)
    path_img = path/'images'
    fnames = get_image_files(path_img)
    pat = pat = r'/([^/]+)_\d+.jpg$'
    data = ImageDataBunch.from_name_re(path_img, fnames, pat, ds_tfms=get_transforms(), size=224, bs=bs
                                  ).normalize(imagenet_stats)
    learn = cnn_learner(data, models.resnet34, metrics=error_rate)
    learn.fit_one_cycle(4)

    linear_output_layer = get_named_module_from_model(learn.model, '1.4')
    return learn, linear_output_layer

In [ ]:
#export
def getFeaturesResNet50(img):
    '''Directly gets a feature vector from the last layer of ResNet50
    without any fine tuning.'''
    
    model = models.resnet50(pretrained=True)
    modules = list(model.children())[:-1]
    resNet50 = nn.Sequential(*modules)
    imgVar = Variable(img)
    featuresVar = resNet50.eval()(imgVar)
    features = featuresVar.data
    
    return features
    

In [ ]:
#export
def trainAsModel():
    '''Fine tunes a pretrained ResNet50 model to detect Google Play store  
    categories of Android screenshots and returns the resulting model in learn 
    as well as the last layer in linear_output_layer for subsequent feature extraction.'''
    
    bs = 16
    np.random.seed(2)
    
    path = "/tf/GP-Labeled-Dataset/"
    data = ImageDataBunch.from_folder(path, ds_tfms=get_transforms(), size=299, bs=bs).normalize(imagenet_stats)
    learn = cnn_learner(data, models.resnet50, metrics=error_rate)
    learn.load('/tf/models/Android_ResNet50_16Epochs')
    
    linear_output_layer = get_named_module_from_model(learn.model, '1.4')
    return learn, linear_output_layer

In [ ]:
#export
class Extractor():
    
    '''Low level implementation of Extractor class for non-layered models only.'''
    
    def __init__(self, learn, linear_output_layer, model, isFT):
        self.models = {'resnet50'}
        self.learn = learn
        self.linear_output_layer = linear_output_layer
        self.model = model
        self.isFT = isFT
        
        if self.model not in self.models:
            raise Exception("Invalid architecture: " + self.model)
    
    def getFeatures(self, img):
        if self.model == 'resnet50':
            resized_img = cv2.resize(img, (224, 224))
        final_img = np.expand_dims(np.transpose(resized_img, [2, 0, 1]), axis=0)
        tensor = torch.from_numpy(final_img)
        
        img_repr = []
        if self.isFT:
            floatTensor = tensor.type(torch.cuda.FloatTensor)
            with Hook(self.linear_output_layer, get_output, True, True) as hook:
                result = self.learn.model.eval()(floatTensor)
                img_repr = hook.stored.cpu().numpy()
        else:
            floatTensor = tensor.type(torch.FloatTensor)
            img_repr = getFeaturesResNet50(floatTensor).numpy().reshape(1, -1)
        
        return img_repr
    

In [ ]:
#export
class LayeredExtractor():
    
    '''Low level implementation of Extractor class for layered models only.'''
    
    def __init__(self):
        self.features = []
        
    def hook_fn(self, module, input, output):
        output_flat = output.detach().flatten()
        self.features = np.hstack((self.features, output_flat))

    def getFeatures(self, img):
        self.features = []
        
        resized_img = cv2.resize(img, (224, 224))
        final_img = np.expand_dims(np.transpose(resized_img, [2, 0, 1]), axis=0)
        tensor = torch.from_numpy(final_img)
        floatTensor = tensor.type(torch.FloatTensor)
        
        model = models.resnet50(pretrained=True)
        for n, m in model.named_modules():
            if 'pool' in n:
                m.register_forward_hook(self.hook_fn)
        model.eval()(floatTensor)
        self.features = self.features.reshape(1, -1)
        return self.features
    

In [ ]:
#export
def createExtractor(learn, linear_output_layer, model, isFT):
    return Extractor(learn, linear_output_layer, model, isFT)
def createLayeredExtractor():
    return LayeredExtractor()

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_prep.ipynb.
Converted 01_features.ipynb.
Converted 02_eval.ipynb.
Converted 03_cnn.ipynb.
Converted index.ipynb.
Converted lesson1-pets.ipynb.
